In [1]:
#importacion de librerias
import pandas as pd
import os 
from pathlib import Path
import utils
from sklearn.model_selection import train_test_split
import spacy

In [2]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.1/12.8 MB 825.8 kB/s eta 0:00:16
     - -------------------------------------- 0.3/12.8 MB 2.5 MB/s eta 0:00:05
     -- ------------------------------------- 0.7/12.8 MB 4.5 MB/s eta 0:00:03
     ---- ----------------------------------- 1.3/12.8 MB 5.5 MB/s eta 0:00:03
     ----- ---------------------------------- 1.9/12.8 MB 6.6 MB/s eta 0:00:02
     ------- -------------------------------- 2.5/12.8 MB 7.5 MB/s eta 0:00:02
     --------- ------------------------------ 3.1/12.8 MB 8.6 MB/s eta 0:00:02
     ----------- ---------------------------- 3.6/12.8 MB 8.2 MB/s eta 0:00:02
     ------------ --------------------------- 4.1/12.8 MB 8.8 MB/s eta 0:00:01
     -------------- ------------------------- 4.5/12.8 MB 8.8 MB/s eta 0:00:01
     --------------- ------------------------ 5.1/12.8 MB 9.1 MB/

In [3]:
#carga de datos 
data_path = Path(os.getcwd()).parent / "data"
train_path = data_path / "bronze" / "dreaddit-train.csv"
test_path = data_path / "bronze" / "dreaddit-test.csv"

train = pd.read_csv(train_path, usecols=['text', 'id', 'label'])
test =  pd.read_csv(test_path, usecols=['text', 'id', 'label'])
df = pd.concat([train,test]).reset_index(drop=True)
df["text"] = df["text"].str.lower()

In [4]:
print(f"Hay {df.shape[0]} registros en el dataset")

Hay 3553 registros en el dataset


In [5]:
#Se filtran los textos con solo una palabra
df["token_count"] = df.text.apply(lambda x: len(x.split()))
ind_tk = df["token_count"] ==1
df = df[~ind_tk]

In [6]:
# Cargar el modelo de SpaCy
nlp = spacy.load('en_core_web_sm')
# se almacenan todos los textos en una lista
texts = df['text'].tolist()

# Procesar los textos en lotes
docs = nlp.pipe(texts)
#Se remueen las stopwords
df["clean_text"]  = [utils.remove_stopwords(doc,lemma=False,remove_stop=True) for doc in docs]

In [7]:

#Se eliminan los carateres especiales
df["clean_text"]= df["text"].apply(utils.clean_text)
#Se eliminan los carateres especiales pero se mantienen los que son cruciales para indicar la separacion de oraciones.
df["clean_text_sentence_sep"]= df["text"].apply(utils.clean_text,keep="sen", remove_words =True)


In [8]:
#Se separan los datos en test y train
df_train, df_test= train_test_split(df, test_size=0.1, random_state=42)

In [9]:
#Se exportan los datos procesados  a la siguiente capa 'silver'
data_path = Path(os.getcwd()).parent / "data"
train_output_path = data_path / "silver" / "dreadditCleanTrain.csv"
test_output_path = data_path / "silver" / "dreadditCleanTest.csv"

df_train.to_csv(train_output_path, index=False)
df_test.to_csv(test_output_path, index= False)